## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-25-21-51-19 +0000,bbc,Trump raises tariffs on Canadian goods in resp...,https://www.bbc.com/news/articles/cx2ljgrm78zo...
1,2025-10-25-21-46-21 +0000,nyt,Neutrogena Recalls Makeup Wipes Over Bacterial...,https://www.nytimes.com/2025/10/25/business/ne...
2,2025-10-25-21-34-39 +0000,bbc,Connolly declared president of Ireland after l...,https://www.bbc.com/news/articles/c4gk2kml122o...
3,2025-10-25-21-30-00 +0000,wsj,Trump Says He Will Hike Tariffs on Canada By 1...,https://www.wsj.com/politics/policy/trump-hike...
4,2025-10-25-21-27-26 +0000,bbc,Louvre moves jewels to ultra-secure Bank of Fr...,https://www.bbc.com/news/articles/crmxkl94w3zo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2385/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,43
21,he,14
287,new,11
22,will,10
24,canada,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
3,2025-10-25-21-30-00 +0000,wsj,Trump Says He Will Hike Tariffs on Canada By 1...,https://www.wsj.com/politics/policy/trump-hike...,91
82,2025-10-25-14-56-21 +0000,nypost,Trump ‘open’ to meeting with Kim Jong Un durin...,https://nypost.com/2025/10/25/world-news/trump...,87
133,2025-10-25-09-38-00 +0000,wsj,Trump Pitches Meeting With Kim Jong Un During ...,https://www.wsj.com/world/asia/trump-pitches-m...,82
216,2025-10-24-22-03-08 +0000,nyt,"Trump Decries Anti-Tariff Ad From Ontario, Can...",https://www.nytimes.com/2025/10/24/us/politics...,82
21,2025-10-25-20-30-41 +0000,nypost,Trump not meeting with Putin again until he kn...,https://nypost.com/2025/10/25/world-news/trump...,81


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
3,91,2025-10-25-21-30-00 +0000,wsj,Trump Says He Will Hike Tariffs on Canada By 1...,https://www.wsj.com/politics/policy/trump-hike...
82,44,2025-10-25-14-56-21 +0000,nypost,Trump ‘open’ to meeting with Kim Jong Un durin...,https://nypost.com/2025/10/25/world-news/trump...
89,37,2025-10-25-13-38-40 +0000,nypost,House Republicans want to strip Zohran Mamdani...,https://nypost.com/2025/10/25/us-news/house-re...
106,34,2025-10-25-12-00-31 +0000,bbc,'I am not done' - Kamala Harris tells BBC she ...,https://www.bbc.com/news/articles/cx2n7k2veywo...
35,33,2025-10-25-19-00-03 +0000,nypost,Republican Jack Ciattarelli scores another Dem...,https://nypost.com/2025/10/25/us-news/republic...
182,31,2025-10-25-01-00-00 +0000,wsj,The pay gap between women and men widened to 8...,https://www.wsj.com/economy/gender-pay-gap-ret...
24,29,2025-10-25-20-15-45 +0000,nypost,Luxury Greenwich Village pad used in NBA poker...,https://nypost.com/2025/10/25/us-news/dozens-o...
81,29,2025-10-25-15-21-49 +0000,nypost,Chilling footage shows Russian drone hunting d...,https://nypost.com/2025/10/25/world-news/chill...
154,27,2025-10-25-05-22-20 +0000,nypost,"Oldest House member Eleanor Holmes Norton, 88,...",https://nypost.com/2025/10/25/us-news/house-me...
168,26,2025-10-25-02-56-31 +0000,nyt,South Korea Tries to Curb Anti-China Protests ...,https://www.nytimes.com/2025/10/24/world/asia/...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
